In [1]:
import pandas as pd
import numpy as np
import json
import os

In [2]:
#Read one json-File
def readOneFile(file):
    with open(file,'r') as filename:
        data = json.load(filename)    
    return data
   

In [3]:
#Read all json-files
def readAllFiles(folder):
    pass    


In [4]:
#readOneFile('all_data_dec_2023_1212/2023_11_18T16_00_00Z.json')

In [5]:
def extractValues(data):
    values = data['results']['A']['frames'][0]['data']['values']
    return values

In [6]:
def extractNames(data):
    names = []
    try:
        fields = data['results']['A']['frames'][0]['schema']['fields']
        for field in fields:
            # Safely accessing the 'name' key from each field dictionary
            name = field.get('name')
            if name is not None:  # Ensuring that 'name' key exists
                names.append(name)
    except (IndexError, KeyError, TypeError) as e:
        print(f"An error occurred while parsing the JSON data: {e}")

    return names

In [7]:
def transposeData(values,names):
    transposed_data = list(map(list, zip(*values)))
    df = pd.DataFrame(transposed_data, columns=names)
    return df

In [8]:
def convertToCsv(df):
    df.to_csv("./all.csv")

In [9]:
# Function to parse and convert the "Angle" column data into a usable DataFrame
def process_angle_data(angle_data):
    angle_parsed = json.loads(angle_data)
    
    # Extract the 'Rows' which contain the torque data
    angle_rows = angle_parsed[0]['Rows']
    
    # Create and return a DataFrame from the extracted rows
    return pd.DataFrame(angle_rows, columns=['Timestamp', 'Angle'])

In [10]:
# Function to parse and convert the "Torgue" column data into a usable DataFrame
def process_torgue_data(torgue_data):
    torgue_parsed = json.loads(torgue_data)[0]  
    
    # Extract the 'Rows' which contain the torque data
    torgue_rows = torgue_parsed['Rows']
    
    # Create and return a DataFrame from the extracted rows
    return pd.DataFrame(torgue_rows, columns=['Timestamp', 'Torque'])

In [11]:
def parse_json(column_data):
    parsed_data = json.loads(column_data)
    rows = []
    for entry in parsed_data:
        measurement_name = entry['Name']
        for timestamp, value in entry['Rows']:
            rows.append({
                'MeasurementName': measurement_name,
                'Timestamp': timestamp,
                entry['Name']: value
            })
    return pd.DataFrame(rows)

def explode_json_within_df(df, json_columns):
    temp_dfs = []
    for index, row in df.iterrows():
        # Parsing JSON data for all specified columns in the row
        expanded_data = [parse_json(row[col]) for col in json_columns]

        # Merging all expanded data on 'Timestamp' column
        combined_expanded_data = expanded_data[0]
        for data in expanded_data[1:]:
            combined_expanded_data = pd.merge(combined_expanded_data, data, on='Timestamp', suffixes=('', '_other'))

        # Repeat the rest of the row data to match the length of the combined expanded data
        repeated_data = pd.DataFrame([row.drop(json_columns)] * len(combined_expanded_data)).reset_index(drop=True)

        # Concatenate the repeated data with the combined expanded data
        final_combined_data = pd.concat([repeated_data, combined_expanded_data], axis=1)

        # Appending each fully processed row to the list
        temp_dfs.append(final_combined_data)

    return pd.concat(temp_dfs, ignore_index=True)


In [12]:
def start(foldername):
    df = pd.DataFrame()
    for file in os.listdir(foldername):
        data = readOneFile(os.path.join(foldername,file))
        values = extractValues(data)
        names = extractNames(data)
        dftemp = transposeData(values,names)
        df = pd.concat([df, dftemp])
    df.to_csv("./"+'all'+".csv")
    return df
        


In [13]:
full_data = start('all_data_dec_2023_1212')

In [14]:
first_rows = full_data[['Angle', 'Torque']].head()
first_rows.to_csv('first_rows_versions.csv')

',Angle,Torque\n0,"[{""Name"":""Angle"",""Fields"":[{""Name"":""timestamp"",""Type"":""time"",""Unit"":""s""},{""Name"":""Angle"",""Type"":""number"",""Unit"":""�""}],""Rows"":[[0,0.3608],[0.009998321533203125,0.47550000000000003],[0.015996932983398438,0.5719000000000001],[0.022001266479492188,0.6893],[0.025999069213867188,0.7787000000000001],[0.029993057250976562,0.877],[0.03399848937988281,0.9848],[0.037998199462890625,1.1021],[0.03999519348144531,1.1641000000000001],[0.043994903564453125,1.2946],[0.04599761962890625,1.3633],[0.05000495910644531,1.5078],[0.051998138427734375,1.5834000000000001],[0.05399322509765625,1.6611],[0.05799293518066406,1.8225],[0.059993743896484375,1.9068],[0.062000274658203125,1.9936],[0.06399726867675781,2.0828],[0.06599617004394531,2.1744],[0.06799507141113281,2.2683],[0.06999778747558594,2.3643],[0.07199478149414062,2.4625],[0.07399749755859375,2.5628],[0.07599449157714844,2.6651000000000002],[0.07799339294433594,2.77],[0.07999610900878906,2.8773],[0.0819

In [35]:
first_rows = explode_json_within_df(full_data[['Angle', 'Torque']].head(5), ['Angle', 'Torque']) 
first_rows


,MeasurementName,Timestamp,Angle,MeasurementName_other,Torque
0,Angle,0.000000,0.3608,Torque,5.9
1,Angle,0.009998,0.4756,Torque,6.1
2,Angle,0.015995,0.5720,Torque,7.1
3,Angle,0.021999,0.6893,Torque,6.4
4,Angle,0.025997,0.7786,Torque,7.4
...,...,...,...,...,...
1663,Angle,3.373995,359.9841,Torque,-2.2
1664,Angle,3.375994,0.0189,Torque,-1.2
1665,Angle,3.377995,0.0516,Torque,-1.1
1666,Angle,3.385998,0.1611,Torque,-1.5


In [29]:
full_data.index.value_counts()

0      427
1      421
2      415
3      414
4      413
      ... 
177      4
178      3
179      3
180      3
181      2
Name: count, Length: 182, dtype: int64